In [ ]:
!pip install line-bot-sdk flask flask-ngrok
# !pip install translate
!pip install --upgrade wenxin-api
!pip install opencc-python-reimplemented
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from flask import Flask, request, abort, jsonify
from flask_ngrok import run_with_ngrok
# from translate import Translator
import wenxin_api
from wenxin_api.tasks.text_to_image import TextToImage
from wenxin_api.tasks.composition import Composition
from wenxin_api.tasks.free_qa import FreeQA

from opencc import OpenCC
import requests
import json

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models.events import FollowEvent
from linebot.models import *
from linebot.models.template import ButtonsTemplate, CarouselTemplate, ConfirmTemplate, ImageCarouselTemplate

In [ ]:
# 文字生圖。目前支持风格有：油画、水彩画、卡通、粉笔画、儿童画、蜡笔画
def text_to_image(input_text, style = "水彩画"):
    input_text = OpenCC('tw2s').convert(input_text)
    input_dict = {
        "text": input_text,
        "style": style
    }
    rst = TextToImage.create(**input_dict)
    return rst['imgUrls']


# 寫文章
def zuowen(input_text):
    input_text = OpenCC('tw2s').convert(input_text)
    input_dict = {
        "text": f"作文题目：{input_text}\n内容：",
        "seq_len": 128,
        "topp": 0.6,
        "penalty_score": 1.2,
        "min_dec_len": 2,
        "min_dec_penalty_text": "。?：！[<S>]",
        "is_unidirectional": 0,
        "task_prompt": "zuowen",
        "mask_type": "paragraph"
    }
    rst = Composition.create(**input_dict)
    return OpenCC('s2tw').convert(rst['result'])


# 自由問答
def qa(input_text):
    input_text = OpenCC('tw2s').convert(input_text)
    input_dict = {
        "text": f"问题：{input_text}\n回答：",
        "seq_len": 128,
        "topp": 0.6,
        "penalty_score": 1.2,
        "min_dec_len": 2,
        "min_dec_penalty_text": "。?：！[<S>]",
        "is_unidirectional": 0,
        "task_prompt": "qa",
        "mask_type": "paragraph"
    }
    rst = FreeQA.create(**input_dict)
    return OpenCC('s2tw').convert(rst['result'])

In [ ]:
# 生成歌詞
def lyric_generate(input_text):
    url = "https://wenxin.baidu.com/wenxin/demo/genlyrics"
    payload = {
        'keyWord': '["' + input_text + '"]',
        'name': '歌词'
    }
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': 'BAIDUID=9B0A87B3977A05C8B4A2D700254754D4:FG=1'
    }

    res = requests.post(url, headers=headers, data=payload)
    result = res.json()['content']['result']
    verse = ""
    chorus = ""
    if '主歌' in result:
        verse = result.split('主歌\n')[-1]
        if '副歌' in verse:
            chorus = OpenCC('s2tw').convert(verse.split('副歌\n')[-1])
            verse = OpenCC('s2tw').convert(verse.split('副歌\n')[0])

    with open("/content/drive/MyDrive/lyric.json", encoding='utf8') as f:
        jd = json.load(f)

    jd[0]['contents']['body']['contents'][0]['text'] = input_text
    jd[0]['contents']['body']['contents'][1]['contents'][0]['contents'][1]['text'] = verse
    jd[0]['contents']['body']['contents'][1]['contents'][1]['contents'][1]['text'] = chorus
    lyric_message = FlexSendMessage.new_from_json_dict(jd[0])
    return lyric_message

In [ ]:
def detect_json_array_to_new_message_array(fileName):
    with open(fileName, encoding='utf8') as f:
        jsonArray = json.load(f)
    
    returnArray = []
    for jsonObject in jsonArray:
        message_type = jsonObject.get('type')
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    

    return returnArray

In [ ]:
list_message = detect_json_array_to_new_message_array("/content/drive/MyDrive/list.json")
text_message1 = TextSendMessage("請用中文輸入prompt，您可以这样描述：圖片主體，細節詞，修飾詞")
text_message2 = TextSendMessage("請用中文輸入文章的關鍵詞")
text_message3 = TextSendMessage("請用中文輸入你的問題")
quickreply_message = detect_json_array_to_new_message_array("/content/drive/MyDrive/quickreply.json")
text_message4 = TextSendMessage("正在創作中，請稍候...")
image_message = detect_json_array_to_new_message_array("/content/drive/MyDrive/img.json")
template_message = detect_json_array_to_new_message_array("/content/drive/MyDrive/temp.json")
flex_message = detect_json_array_to_new_message_array("/content/drive/MyDrive/flex.json")


# message_dict = {
#     "我想要文字生圖": text_message1,
#     "我想要生成文章": text_message2,
#     "我想要自由問答": lyric_message,
#     "我想要風格轉換": quickreply_message
# }

In [ ]:
# def translate(content, from_lang, to_lang):
#     translator= Translator(from_lang = from_lang, to_lang = to_lang)
#     translation = translator.translate(content)
#     return translation

In [ ]:
app = Flask(__name__, static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)

line_bot_api = LineBotApi("your channel access token")
handler = WebhookHandler("your channel secret")
wenxin_api.ak = "your api key"
wenxin_api.sk = "your secret key"


@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text = True)
    print(body)

    with open("/content/drive/MyDrive/ai-event.log", "a") as f:
        f.write(body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
with open("/content/drive/MyDrive/menu.json", encoding='utf8') as f:
    menuJson = json.load(f)

lineRichMenuId = line_bot_api.create_rich_menu(rich_menu = RichMenu.new_from_json_dict(menuJson))
uploadImageFile = open("/content/drive/MyDrive/menu.jpg", 'rb')
setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId, 'image/jpeg', uploadImageFile)


@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    with open("/content/drive/MyDrive/users.txt", "a") as f:
        f.write(json.dumps(vars(user_profile), sort_keys=True))
        f.write('\n')

    linkResult = line_bot_api.link_rich_menu_to_user(user_profile.user_id, lineRichMenuId)
    line_bot_api.reply_message(
        event.reply_token,
        list_message            # [TextSendMessage(f'你好 {user_profile.display_name}，歡迎加入社群')]
    )

In [ ]:
# @handler.add(MessageEvent, message = TextMessage)
# def handle_message(event):
#     if (event.message.text.find('@') != -1):
#         line_bot_api.reply_message(
#         event.reply_token,
#         message_dict.get(event.message.text)
#         )
#     else:
#         line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text = event.message.text)
#         )

In [ ]:
@handler.add(MessageEvent, message = TextMessage)
def handle_message(event):
    # if (event.message.text.find('文字生圖') != -1):
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     text_message1)     # message_dict.get(event.message.text)
    # elif (event.message.text.find('生成文章') != -1):
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     text_message2)
    # elif (event.message.text.find('自由問答') != -1):
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     lyric_message) 
    # elif (event.message.text.find('風格轉換') != -1):
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     quickreply_message)  
    # else:
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     TextSendMessage(text = qa(event.message.text)))
    line_bot_api.reply_message(
    event.reply_token,
    lyric_generate(event.message.text))
    

In [ ]:
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('@文字生圖') == 0):
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='請稍待，文字生圖'))
    elif (event.postback.data.find('@生成文章') == 0):
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='請稍待，生成文章'))
    else:
        pass

In [ ]:
@handler.add(MessageEvent, message = ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='圖片已收到，正在努力創作中，請稍候...'))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as f:
        for chunk in message_content.iter_content():
            f.write(chunk)

In [ ]:
@handler.add(MessageEvent, message = VideoMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp4', 'wb') as f:
        for chunk in message_content.iter_content():
            f.write(chunk)

In [ ]:
@handler.add(MessageEvent, message = AudioMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Audio has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp3', 'wb') as f:
        for chunk in message_content.iter_content():
            f.write(chunk)

In [ ]:
# 指定推播
# line_bot_api.push_message('U64e856315f80175e21257c6e7eb77412', TextSendMessage(text = 'Hello World!'))

# 廣播
# json_object_strings = open("/content/drive/MyDrive/users.txt", 'r')
# json_array = []
# user_id_array = []

# for json_object_string in json_object_strings:
#     json_object = json.loads(json_object_string)
#     json_array.append(json_object)

# for user_record in json_array:
#     user_id_array.append(user_record.get("user_id"))

# line_bot_api.multicast(
#     user_id_array,
#     TextSendMessage(text = 'Hello World!')
# )

In [ ]:
app.run()

In [ ]:
# !pip install paddlepaddle-gpu==2.3.2

In [ ]:
# !git clone https://github.com/wzmsltw/PaintTransformer

In [ ]:
# !cd content/PaintTransformer
# !mv [Download Directory]/paint_best.pdparams inference/

In [ ]:
# from PaintTransformer.inference import inference
# !python inference.py